In [155]:
!pip install --upgrade pip
!pip install pyarrow
!pip install pandas
!pip install duckdb
!pip install hnswlib

In [156]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import duckdb
import hnswlib

In [157]:
test_table = pq.read_table('test_data_to_record.parquet')
panda_test_table = test_table.to_pandas()
# print(panda_test_table.head())

train_table = pq.read_table('train_data_to_record.parquet')
panda_train_table = train_table.to_pandas()
print(panda_train_table.head())


                                      embedding_data  \
0  [-19.19756317138672, -18.10773468017578, -22.2...   
1  [6.993745803833008, -16.4917049407959, -12.157...   
2  [-13.4398193359375, -6.445910930633545, -7.608...   
3  [-15.908609390258789, 7.963416576385498, -12.5...   
4  [-16.739404678344727, -16.108036041259766, -20...   

                resource_uri                                      metadata  \
0  train-images-idx3-ubyte-0  {'location': 'San Francisco', 'quality': 85}   
1  train-images-idx3-ubyte-1         {'location': 'Chicago', 'quality': 0}   
2  train-images-idx3-ubyte-2  {'location': 'San Francisco', 'quality': 35}   
3  train-images-idx3-ubyte-3   {'location': 'San Francisco', 'quality': 1}   
4  train-images-idx3-ubyte-4         {'location': 'Dallas', 'quality': 58}   

                                   infer  \
0  {'annotations': [{'category_id': 5}]}   
1  {'annotations': [{'category_id': 0}]}   
2  {'annotations': [{'category_id': 4}]}   
3  {'annotations':

In [158]:
# print(duckdb.query('''
# SELECT COUNT(*)
# FROM 'test_data_to_record.parquet'
# WHERE metadata.location ILIKE '%San Francisco%';
# ''').fetchall())
print("Record in test")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'test_data_to_record.parquet'
;
''').fetchall())
print("Record in train")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'train_data_to_record.parquet'
;
''').fetchall())

Record in test
[(10000,)]
Record in train
[(60000,)]


In [159]:
import hnswlib
import numpy as np

#dim = 16
# num_elements = 10000

# Generating sample data
# data = np.float32(np.random.random((num_elements, dim)))

# We split the data in two batches:
data1 = data[:num_elements // 2]
data2 = data[num_elements // 2:]

data1 = panda_test_table['embedding_data'].to_numpy().tolist()
data2 = panda_train_table['embedding_data'].to_numpy().tolist()
dim = len(data1[0])
num_elements = len(data1) + len(data2)
# print("dimensionality is:", dim)
# print("total number of elements is:", num_elements)
# print("max elements", num_elements//2)

concatted_data = data1 + data2
# print("concatted_data", len(concatted_data))

# Declaring index
p = hnswlib.Index(space='l2', dim=dim)  # possible options are l2, cosine or ip

# Initing index
# max_elements - the maximum number of elements (capacity). Will throw an exception if exceeded
# during insertion of an element.
# The capacity can be increased by saving/loading the index, see below.
#
# ef_construction - controls index search speed/build speed tradeoff
#
# M - is tightly connected with internal dimensionality of the data. Strongly affects the memory consumption (~M)
# Higher M leads to higher accuracy/run_time at fixed ef/efConstruction

p.init_index(max_elements=len(data1), ef_construction=100, M=16)

# Controlling the recall by setting ef:
# higher ef leads to better accuracy, but slower search
p.set_ef(10)

# Set number of threads used during batch search/construction
# By default using all available cores
p.set_num_threads(4)

# print(data1[0])

# print("Adding first batch of %d elements" % (len(data1)))
p.add_items(data1)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(data1, k=1)
print(labels)

print(distances)
# print("Recall for the first batch:", np.mean(labels.reshape(-1) == np.arange(len(data1))), "\n")

# Serializing and deleting the index:
index_path='first_half.bin'
# print("Saving index to '%s'" % index_path)
p.save_index(index_path)
del p

# Reiniting, loading the index
p = hnswlib.Index(space='l2', dim=dim)  # the space can be changed - keeps the data, alters the distance function.

# print("\nLoading index from 'first_half.bin'\n")

# Increase the total capacity (max_elements), so that it will handle the new data
p.load_index("first_half.bin", max_elements = num_elements)

# print("Adding the second batch of %d elements" % (len(data2)))
p.add_items(data2)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(concatted_data, k=1)
# print("Recall for two batches:", np.mean(labels.reshape(-1) == np.arange(len(concatted_data))), "\n")

[[   0]
 [   1]
 [   2]
 ...
 [9997]
 [9998]
 [9999]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [160]:
print("Asdf")

Asdf


In [169]:
def unpack_annotations(embeddings):
    # Get and unpack inference data
    annotations = [
        embedding['infer']["annotations"]
        for embedding in embeddings
    ]  # Unpack JSON
    # print("annotations", annotations)
    annotations = [
        annotation for annotation_list in annotations for annotation in annotation_list
    ]  # Flatten the list
    # print("annotations", annotations)

    # categories_by_uid = {
    #     annotation["id"]: annotation["category_id"] for annotation in annotations
    # }
    # print(categories_by_uid)

    # # Unpack embedding data
    embeddings = [embedding["embedding_data"] for embedding in embeddings]

    embedding_vectors_by_category = {}
    for embedding_annotation_pair in zip(embeddings, annotations):
        data = np.array(embedding_annotation_pair[0])
        category = embedding_annotation_pair[1]['category_id'] #categories_by_uid[embedding["target"]]
        if category in embedding_vectors_by_category.keys():
            embedding_vectors_by_category[category] = np.append(
                embedding_vectors_by_category[category], data[np.newaxis, :], axis=0
            )
        else:
            embedding_vectors_by_category[category] = data[np.newaxis, :]

    return embedding_vectors_by_category

# Get the training embeddings. This is the set of embeddings belonging to datapoints of the training dataset, and to the first created embedding set.
training_embedding_vectors_by_category = unpack_annotations(panda_train_table.to_dict('records'))

# print("training_embedding_vectors_by_category", training_embedding_vectors_by_category)
print("panda_test_table", panda_test_table.head())



panda_test_table                                       embedding_data  \
0  [-12.84811019897461, -15.020103454589844, -10....   
1  [-8.334704399108887, -6.763201713562012, 6.999...   
2  [-12.62231159210205, 6.12582540512085, -9.9566...   
3  [5.2354021072387695, -13.995504379272461, -11....   
4  [-11.92990493774414, -14.994053840637207, -12....   

               resource_uri                                 metadata  \
0  t10k-images-idx3-ubyte-0   {'location': 'Chicago', 'quality': 26}   
1  t10k-images-idx3-ubyte-1   {'location': 'Atlanta', 'quality': 82}   
2  t10k-images-idx3-ubyte-2  {'location': 'New York', 'quality': 38}   
3  t10k-images-idx3-ubyte-3  {'location': 'New York', 'quality': 67}   
4  t10k-images-idx3-ubyte-4   {'location': 'Chicago', 'quality': 92}   

                                   infer  \
0  {'annotations': [{'category_id': 7}]}   
1  {'annotations': [{'category_id': 2}]}   
2  {'annotations': [{'category_id': 1}]}   
3  {'annotations': [{'category_id': 0

In [162]:
inv_covs_by_category = {}
means_by_category = {}
for category, embeddings in training_embedding_vectors_by_category.items():
    print(f"Computing mean and covariance for label category {category}")

    # Compute the mean and inverse covariance for computing MHB distance
    print(f"category: {category} samples: {embeddings.shape[0]}")
    if embeddings.shape[0] < (embeddings.shape[1] + 1):
        print(f"not enough samples for stable covariance in category {category}")
        continue
    cov = np.cov(embeddings.transpose())
    try:
        inv_cov = np.linalg.inv(cov)
    except np.linalg.LinAlgError as err:
        print(f"covariance for category {category} is singular")
        continue
    mean = np.mean(embeddings, axis=0)
    inv_covs_by_category[category] = inv_cov
    means_by_category[category] = mean

# print(means_by_category[5])
target_datapoints = panda_test_table.to_dict('records') + panda_train_table.to_dict('records')
# print(target_datapoints)

# Process each datapoint's inferences individually. This is going to be very slow.
# This is because there is no way to grab the corresponding metadata off the datapoint.
# We could instead put it on the embedding directly ?
inference_metadata = {}
for idx, datapoint in enumerate(target_datapoints):
    inferences = datapoint['infer']["annotations"]
    embeddings = [datapoint["embedding_data"]]

    for i in range(len(inferences)):
        emb_data = embeddings[i]
        category = inferences[i]["category_id"]
        if not category in inv_covs_by_category.keys():
            continue
        mean = means_by_category[category]
        inv_cov = inv_covs_by_category[category]
        delta = np.array(emb_data) - mean
        squared_mhb = np.sum((delta * np.matmul(inv_cov, delta)), axis=0)
        if squared_mhb < 0:
            print(f"squared distance for category {category} is negative")
            continue
        distance = np.sqrt(squared_mhb)
        # print(datapoint)
        inference_metadata[datapoint["resource_uri"]] = distance

# dict to json
import json

# quality scores from db verison 0.0.2
# 1|{"distance_score": 3.0529832292425403}
# 2|{"distance_score": 2.088222399506212}
# 3|{"distance_score": 3.7741301182872675}
# 4|{"distance_score": 2.3315582411124463}
# 5|{"distance_score": 1.6887465106278159}
# 6|{"distance_score": 2.1548636869467788}
# 7|{"distance_score": 3.2068276867648056}
# 8|{"distance_score": 2.3359545036332805}
# 9|{"distance_score": 2.4505819331999517}

# id|created_at|updated_at|data|input_identifier|inference_identifier|label|embedding_set_id|layer_id|datapoint_id
# 1|2022-10-13 16:48:50|2022-10-13 16:48:50|[-19.19756317138672, -18.10773468017578, -22.252187728881836, -1.7129746675491333, -20.99203109741211, 4.6510820388793945, -11.819682121276855, -17.237871170043945, -10.40627384185791, -7.90831184387207]||||1||1
# 2|2022-10-13 16:48:50|2022-10-13 16:48:50|[6.993745803833008, -16.4917049407959, -12.157032012939453, -21.126211166381836, -19.217905044555664, -21.072765350341797, -9.723943710327148, -14.83087158203125, -14.961665153503418, -14.060187339782715]||||1||2
# 3|2022-10-13 16:48:50|2022-10-13 16:48:50|[-13.4398193359375, -6.445910930633545, -7.608524799346924, -13.623757362365723, 5.8722381591796875, -15.087124824523926, -12.686633110046387, -8.033711433410645, -11.658220291137695, -8.04366683959961]||||1||3
# 4|2022-10-13 16:48:50|2022-10-13 16:48:50|[-15.908609390258789, 7.963416576385498, -12.563691139221191, -18.787290573120117, -7.542619705200195, -12.971125602722168, -12.110857963562012, -9.590605735778809, -12.207917213439941, -17.283729553222656]||||1||4
# 5|2022-10-13 16:48:50|2022-10-13 16:48:50|[-16.739404678344727, -16.108036041259766, -20.445873260498047, -16.767101287841797, -7.684770584106445, -16.530820846557617, -23.035736083984375, -13.244646072387695, -9.138373374938965, 5.229954719543457]||||1||5
# 6|2022-10-13 16:48:50|2022-10-13 16:48:50|[-16.288536071777344, -18.42648696899414, 7.389773368835449, -15.272856712341309, -18.917083740234375, -25.22352409362793, -19.038244247436523, -17.658218383789062, -8.270002365112305, -19.835947036743164]||||1||6
# 7|2022-10-13 16:48:50|2022-10-13 16:48:50|[-13.312373161315918, 6.984036922454834, -13.242178916931152, -9.999306678771973, -6.004954814910889, -10.29246997833252, -13.447978973388672, -8.89227294921875, -8.559014320373535, -10.059027671813965]||||1||7
# 8|2022-10-13 16:48:50|2022-10-13 16:48:50|[-23.386146545410156, -19.125940322875977, -13.216439247131348, 6.500119686126709, -24.382715225219727, -14.136991500854492, -26.391183853149414, -12.205113410949707, -14.392107009887695, -11.659762382507324]||||1||8
# 9|2022-10-13 16:48:50|2022-10-13 16:48:50|[-13.129637718200684, 7.2978363037109375, -15.127612113952637, -10.571370124816895, -4.537073135375977, -8.433134078979492, -12.171133041381836, -7.148005485534668, -9.562264442443848, -9.543116569519043]||||1||9


# print(len(inference_metadata))
print("train-images-idx3-ubyte-0", (inference_metadata["train-images-idx3-ubyte-0"]))
print("train-images-idx3-ubyte-1", (inference_metadata["train-images-idx3-ubyte-1"]))
print("train-images-idx3-ubyte-2", (inference_metadata["train-images-idx3-ubyte-2"]))
print("train-images-idx3-ubyte-3", (inference_metadata["train-images-idx3-ubyte-3"]))

Computing mean and covariance for label category 5
category: 5 samples: 5419
Computing mean and covariance for label category 0
category: 0 samples: 5932
Computing mean and covariance for label category 4
category: 4 samples: 5845
Computing mean and covariance for label category 1
category: 1 samples: 6741
Computing mean and covariance for label category 9
category: 9 samples: 5937
Computing mean and covariance for label category 2
category: 2 samples: 5953
Computing mean and covariance for label category 3
category: 3 samples: 6127
Computing mean and covariance for label category 6
category: 6 samples: 5919
Computing mean and covariance for label category 7
category: 7 samples: 6279
Computing mean and covariance for label category 8
category: 8 samples: 5848
train-images-idx3-ubyte-0 3.4324716467737963
train-images-idx3-ubyte-1 2.208093177145872
train-images-idx3-ubyte-2 3.4119725786935735
train-images-idx3-ubyte-3 2.4228611160906492
